# Extract deep coadds in ra-dec coordinates

 work with Weakly_2024_07
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/12/23
- update : 2024/03/01


In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! eups list -s lsst_distrib

In [ ]:
import lsst.daf.butler as dafButler
import lsst.daf.base as dafBase

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from astropy.visualization import imshow_norm, MinMaxInterval, SqrtStretch
# Astropy
from astropy.visualization import ZScaleInterval, AsinhStretch

from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
from astropy.wcs import WCS


plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom
import pprint

In [ ]:
#Set the matplotlib plot color table.
plt.style.use('tableau-colorblind10')
import seaborn as sns

In [ ]:
# Let us also set `lsst.afw.display` to use the `matplotlib` backend:
afwDisplay.setDefaultBackend('matplotlib')
plt.rcParams['figure.figsize'] = (6.0, 6.0)

In [ ]:
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy import coordinates
import astropy.units as u

In [ ]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

## Give the collection

In [ ]:
#my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

## Config

- 5615 - OK , Not in our galaxy
- 5616 - Non, 1 nuit
- 5839 - Non, 1 nuit
- 5634 - OK - Galaxie

In [ ]:
#all_tracts = np.array([5616, 5634, 5615, 10644, 383, 8188, 3863, 455, 452, 5839, 10643, 382])
all_tracts = [382,383,452,455,
                     3863,3864,
                     5614,5615,5616,5634,5839,
                     #8188,
                     10643,10644]

## List of deep coadds

In [ ]:
datasetRefs = registry.queryDatasets(datasetType='deepCoadd', collections=my_collection, where= "instrument='LATISS'")
for i, ref in enumerate(datasetRefs):
    print(f"========({i})================datasetType = deepCoadd ============================================")
    print("fullId..................:",ref.dataId.full)
    print("band....................:",ref.dataId["band"])
    print("skymap..................:",ref.dataId["skymap"])
    print("tract..................:",ref.dataId["tract"])
    print("patch..................:",ref.dataId["patch"])
    print("run.....................:",ref.run)

## Select a tract and a band

In [ ]:
index_sel = 5
tract_sel = all_tracts[index_sel]
band_sel = "y"

## Load deepCoadd

In [ ]:
datasetRefs = registry.queryDatasets(datasetType='deepCoadd', collections=my_collection, where= "instrument='LATISS'")

In [ ]:
datasetRefs = registry.queryDatasets(datasetType='deepCoadd', collections=my_collection, where= "instrument='LATISS'")
all_dataId = []
all_deepcoadds  = []
all_titles = []
all_patches = []
all_deepCoadd_wcs = []
all_deepCoadd_bbox = []
all_deepCoadd_corners_ra= []
all_deepCoadd_corners_dec = []
all_xvals = []
all_yvals = []
all_str_tract_patch = []

for i, ref in enumerate(datasetRefs):

    band = ref.dataId["band"]
    tract = ref.dataId["tract"]
    patch = ref.dataId["patch"]
    skymap = ref.dataId["skymap"]

    if band == band_sel and tract == tract_sel:
        print(f"========({i})================datasetType = deepCoadd ============================================")
        print("fullId..................:",ref.dataId.full)
        print("band....................:",ref.dataId["band"])
        print("skymap..................:",ref.dataId["skymap"])
        print("tract..................:",ref.dataId["tract"])
        print("patch..................:",ref.dataId["patch"])
        print("run.....................:",ref.run)
    
    
        the_dataId = {'skymap':skymap,'band': band, 'tract': tract, 'patch': patch}
        the_title = f"Auxtel deepCoadd : band={band}, tract={tract}, patch={patch}" 
        my_deepCoadd = butler.get('deepCoadd', dataId=ref.dataId.full,collections=my_collection)

        #my_deepCoadd_wcs = butler.get('deepCoadd.wcs', dataId=ref.dataId.full)
        #my_deepCoadd_bbox = butler.get('deepCoadd.bbox', dataId=ref.dataId.full)
        my_deepCoadd_wcs = my_deepCoadd.wcs
        my_deepCoadd_bbox = my_deepCoadd.getBBox() 
        # get the corner
        my_deepCoadd_corners_ra, my_deepCoadd_corners_dec = get_corners_radec(my_deepCoadd_wcs, my_deepCoadd_bbox)
        my_xvals = [my_deepCoadd_corners_ra[0], my_deepCoadd_corners_ra[1], my_deepCoadd_corners_ra[2], \
             my_deepCoadd_corners_ra[3], my_deepCoadd_corners_ra[0]]
        my_yvals = [my_deepCoadd_corners_dec[0], my_deepCoadd_corners_dec[1], my_deepCoadd_corners_dec[2], \
             my_deepCoadd_corners_dec[3], my_deepCoadd_corners_dec[0]]

        str_tract_patch = '(' + str(ref.dataId['tract']) + ', ' + str(ref.dataId['patch'])+')'
        
        all_dataId.append(the_dataId) 
        all_patches.append(ref.dataId["patch"])
        all_deepcoadds.append(my_deepCoadd)
        all_titles.append(the_title)
        all_deepCoadd_wcs.append(my_deepCoadd_wcs)
        all_deepCoadd_bbox.append(my_deepCoadd_bbox)
        all_deepCoadd_corners_ra.append(my_deepCoadd_corners_ra)
        all_deepCoadd_corners_dec.append(my_deepCoadd_corners_dec)
        all_xvals.append(my_xvals)
        all_yvals.append(my_yvals)
        all_str_tract_patch.append(str_tract_patch)  
    else:
        pass
    

## Summaries

In [ ]:
N = len(all_deepcoadds)
for index in range(N):
    print(index,all_titles[index])

In [ ]:
all_patches = np.array(all_patches)
list_patches = np.unique(all_patches)

In [ ]:
all_dataId

## Plot patchs

In [ ]:
nPatches = len(list_patches)
sns.color_palette("hls", nPatches)
cmap = ListedColormap(sns.color_palette("hls", nPatches))

In [ ]:
patch_to_color = {}
patch_flagplot = {}
for idx,patch in enumerate(list_patches):
    patch_to_color[patch] = cmap.colors[idx]
    patch_flagplot[patch] = False

In [ ]:
fig = plt.figure(figsize=(8, 8))

for idx,dataid in enumerate(all_dataId):  
    patch = dataid['patch']
    if not patch_flagplot[patch]:
        xvals = all_xvals[idx]
        yvals = all_yvals[idx]

        xvals = np.array(all_xvals[idx])
        yvals = np.array(all_yvals[idx])
        xmean = xvals.mean()
        ymean = yvals.mean()
        
        str_tract_patch = all_str_tract_patch[idx]
        color = patch_to_color[patch]
        plt.plot(xvals, yvals, ls='solid', lw=2, color=color,label=str_tract_patch)
        patch_flagplot[patch] = True
        plt.text(xmean,ymean, str(patch),
        verticalalignment='center', horizontalalignment='center', 
        color=color,size=14)

plt.xlabel('RA')
plt.ylabel('Dec')
plt.legend(bbox_to_anchor=(1.1, 1.05), ncol=1)
the_title_tract = f"Auxtel : tract {tract_sel}"
plt.title(the_title_tract)
plt.gca().set_aspect('equal')
plt.show()

## Sky map

https://community.lsst.org/t/how-to-make-a-loop-over-tracts-and-patches-to-split-the-dc2-dataset/6081

### Region

https://astropy-regions.readthedocs.io/en/latest/index.html

In [ ]:
all_deepCoadd_corners_ra = np.array(all_deepCoadd_corners_ra)
all_deepCoadd_corners_dec = np.array(all_deepCoadd_corners_dec)

ll=np.array([all_deepCoadd_corners_ra.min(),all_deepCoadd_corners_dec.min()])
ul=np.array([all_deepCoadd_corners_ra.min(),all_deepCoadd_corners_dec.max()])
ur=np.array([all_deepCoadd_corners_ra.max(),all_deepCoadd_corners_dec.max()])
lr = np.array([all_deepCoadd_corners_ra.max(),all_deepCoadd_corners_dec.min()])

In [ ]:
corners = np.array([ (ll[0], ll[1]), 
                    (ul[0], ul[1] ), 
                    (ur[0], ur[1]), 
                    (lr[0], lr[1])])


In [ ]:
corners

In [ ]:
polygon = Polygon(corners,closed=True)

In [ ]:
p = PatchCollection([polygon], cmap=matplotlib.cm.jet, alpha=0.4)

In [ ]:
from astropy.visualization.wcsaxes import Quadrangle
from matplotlib.patches import Rectangle

In [ ]:
gal_long = np.linspace(-180.,180,360)
gal_lat = np.zeros((360))

In [ ]:
coordinates_galactic_planes = SkyCoord(l=gal_long*u.degree, b=gal_lat*u.degree, frame='galactic')
gp_radec = coordinates_galactic_planes.transform_to('icrs')  

In [ ]:
gp_ra_toplot = coordinates.Angle(gp_radec.ra.degree*u.degree)
gp_ra_toplot = gp_ra_toplot.wrap_at(180*u.degree)

In [ ]:
tract_ra_toplot=coordinates.Angle(corners[:,0]*u.degree)
tract_ra_toplot = tract_ra_toplot.wrap_at(180*u.degree)
tract_dec_toplot=coordinates.Angle(corners[:,1]*u.degree)

In [ ]:
fig = plt.figure(figsize=(10,8))
#ax = fig.add_subplot(111, projection="aitoff")
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(gp_ra_toplot.radian, gp_radec.dec.radian,c="r",label="Galactic Plane",s=1)
#df_sel.plot.scatter(x='RA',y='DEC',c='tract',colormap='tab20',grid=True,ax=ax,figsize=(10,8),title='AUXTEL photometry 2023-2024')
ax.scatter(tract_ra_toplot.radian, tract_dec_toplot.radian,c="b",label=f"tract {tract_sel}",
           s=10)
ax.legend()
ax.grid()
r = Quadrangle((0.0, 0)*u.deg, 0.3*u.deg, 0.15*u.deg,
               edgecolor='green', facecolor='blue',
               transform=ax.get_transform())
#r = Rectangle((3.14, 0.1), 0.5, 0.5,
#              label='Rectangle', edgecolor='red', facecolor='green', linestyle='--',
#              transform=ax.get_transform())
ax.add_patch(r)
plt.suptitle(my_collection)
plt.show()

## plot deepCoadd

In [ ]:
N = len(all_deepcoadds)

for index in range(N):
    the_dataid = all_dataId[index]
    the_deepcoadd = all_deepcoadds[index]
    the_title = f"{index})  "+all_titles[index]
    wcs = all_deepCoadd_wcs[index]

    fitsMd = wcs.getFitsMetadata()
    WCSfMd = WCS(fitsMd)
  
    
    # Plot the calexp we just retrieved
    plt.figure(figsize=(10,10))
    plt.subplot(projection= WCSfMd)
    
    transform = AsinhStretch() + ZScaleInterval()
    scaledImage = transform(the_deepcoadd.image.array)
    #scaledImage = np.flipud(scaledImage)
    plt.imshow(scaledImage,origin="lower",aspect='equal')
    plt.suptitle(the_title,fontsize=10)

    ax = plt.gca()
    overlay = ax.get_coords_overlay('icrs')
    overlay.grid(color='white', ls='dotted')
    #plt.tight_layout()
    plt.show()
